In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms


base_dir = '/Users/yishai/Desktop/GenerativeTabularFusion'
import os
os.chdir(base_dir)    
from app.dotenv import base_dir, data_dir








In [2]:
########## CONFIGURATION ##########
# batch_size = 64
# VISIBLE_UNITS = 784  # 28 x 28 images
# HIDDEN_UNITS = 128
CD_K = 2
# EPOCHS = 10

# DATA_FOLDER = 'data/mnist'
batch_size = int(os.getenv('batch_size'))# batch size
n_epochs = int(os.getenv('n_epochs')) # number of epochs
lr = float(os.getenv('lr')) # learning rate
n_hid = int(os.getenv('n_hid')) # number of neurons in the hidden layer
n_vis = int(os.getenv('n_vis')) # input size. i.e number of neurons in the visible layer

print(f'batch_size: {batch_size}')
print(f'n_epochs: {n_epochs}')
print(f'lr: {lr}')
print(f'n_hid: {n_hid}')
print(f'n_vis: {n_vis}')
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0


if CUDA:
    torch.cuda.set_device(CUDA_DEVICE)

batch_size: 64
n_epochs: 10
lr: 0.01
n_hid: 128
n_vis: 784


In [3]:
########## LOADING DATASET ##########
print('Loading dataset...')

from app.load_data.MNIST import data_loaders
train_loader, test_loader = data_loaders(batch_size,data_dir)


Loading dataset...
train size:  60000 test size:  10000
batch size: torch.Size([64, 1, 28, 28])


In [4]:
########## TRAINING RBM ##########
print('Training RBM...')
# from rbm import RBM

from app.rbm.model import RBM_v3

RBM = RBM_v3
# print(RBM)    

rbm = RBM(n_vis, n_hid, CD_K, use_cuda=CUDA)

for epoch in range(n_epochs):
    epoch_error = 0.0

    for batch, _ in train_loader:
        batch = batch.view(len(batch), n_vis)  # flatten input data

        if CUDA:
            batch = batch.cuda()

        batch_error = rbm.contrastive_divergence(batch)

        epoch_error += batch_error

    print('Epoch Error (epoch=%d): %.4f' % (epoch, epoch_error))

Training RBM...


Epoch Error (epoch=0): 5792716.5000


KeyboardInterrupt: 

In [6]:
########## EXTRACT FEATURES ##########
print('Extracting features...')

train_features = np.zeros((len(train_loader.dataset), n_hid))
train_labels = np.zeros(len(train_loader.dataset))
test_features = np.zeros((len(test_loader.dataset), n_hid))
test_labels = np.zeros(len(test_loader.dataset))

for i, (batch, labels) in enumerate(train_loader):
    batch = batch.view(len(batch), n_vis)  # flatten input data

    if CUDA:
        batch = batch.cuda()

    train_features[i*batch_size:i*batch_size+len(batch)] = rbm.sample_hidden(batch).cpu().numpy()
    train_labels[i*batch_size:i*batch_size+len(batch)] = labels.numpy()

for i, (batch, labels) in enumerate(test_loader):
    batch = batch.view(len(batch), n_vis)  # flatten input data

    if CUDA:
        batch = batch.cuda()

    test_features[i*batch_size:i*batch_size+len(batch)] = rbm.sample_hidden(batch).cpu().numpy()
    test_labels[i*batch_size:i*batch_size+len(batch)] = labels.numpy()

Extracting features...


In [10]:

########## CLASSIFICATION ##########
print('Classifying...')

clf = LogisticRegression()
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)

print('Result: %d/%d;%f' % (sum(predictions == test_labels), test_labels.shape[0], sum(predictions == test_labels)/test_labels.shape[0]))

Classifying...


/Users/yishai/Desktop/GenerativeTabularFusion/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Result: 9314/10000;0
Result: 9314/10000;0.931400
Result: 9314/10000;0
